In [15]:
%matplotlib inline

In [1]:

from __future__ import division
import math
import numpy
import scipy

from scipy import integrate

import matplotlib.pyplot as plt

import ipywidgets as widgets

from ipywidgets import interact, interactive, fixed, interact_manual, Label, Layout

import matplotlib.pylab as pylab
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (14, 6),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
pylab.rcParams.update(params)


In [17]:
# A) Initialisation des variables  #####################################################################

# 1. Constantes physiques (NIST)
h    = 6.626068e-34;  # [J.S]      Planck constant
heV  = 4.135667e-15;  # [eV.s]     Planck constant
hb   = h/(2*math.pi);      # [J.s]      h barre
hbeV = heV/(2*math.pi);    # [eV.s]     h barre
me   = 9.109381e-31;  # [kg]       electron mass
q    = 1.6021764e-19; # [C]        electron charge
c    = 299792458;     # [m.s^(-1)] speed of light in vacuum

# 2. Propriété de la particule étudiée
m = me;           # [kg] masse (me = masse de l'électron)

# 3. Propriétés de l'espace étudié (puits, maille,...)
xMIN = -5e-9 # [m] extension maximale vers la gauche
xMAX = -xMIN # [m] extension maximale vers la droite
xNum = 401   #  [-] nombre de valeurs discrétisées de x (résolution) doit être impair
    

# 4. Niveaux énergétiques et fonctions d'onde
vectEn   = numpy.arange(0,10,1) # nombre niveaux énergétiques sélectionnés pour le plot
vectPSIn = numpy.arange(0,5,1)  # nombre fonctions d'ondes sélectionnées pour le plot

# 5 Définitions de constantes internes
dx    = (xMAX-xMIN)/(xNum-1) # [m]
xArr  = numpy.linspace(xMIN, xMAX, xNum-1)
xOnes = [1.0] * (xNum-1)


In [18]:
# B) Définition du potentiel ####################################################################################

def Potentiel(idxPot,PotMAX,Width,K,n):
    """"Args:  
            - idxPot: entier pour la sélection du potentiel 
            - PotMax: hauteur, 
            - Width: largeur, 
            - K: constante de raideur
            - n: périodicité du sinus
          Return:
            - Pot: le potentiel"""

    # definition du potentiel
    if idxPot==2:
        PotMAX = PotMAX*q
        Pot1 = [-PotMAX/2] * int((xNum-1)/2)
        Pot2 = [PotMAX/2] * int((xNum-1)/2)
        Pot = Pot1 + Pot2
    elif idxPot==3:
        PotMAX = PotMAX*q
        Width = Width*1e-9
        Pot = []
        for ii, xx in enumerate(xArr):
            if (xx <= - Width/2) or (xx >= Width/2):
                Pot.append(PotMAX)
            else:
                Pot.append(0.0)
    elif idxPot==4:
        Pot = 0.52*K*numpy.square(xArr)
        PotMAX = numpy.max(Pot)
    elif idxPot==5:
        PotMAX = PotMAX*q
        Width = Width*1e-9
        Pot = []
        for ii, xx in enumerate(xArr):
            if (xx <= - Width/2) or (xx >= Width/2):
                Pot.append(0.0)
            else:
                Pot.append(PotMAX)
    elif idxPot==6:
        offset = 0.25 # décalage exprimé en périodes (défaut : 0.25)
        PotMAX = PotMAX*q
        Pot = PotMAX * numpy.sin(n * (xArr-xMIN) * (2*math.pi)/(xMAX-xMIN) - offset*(2*math.pi))
    else: 
        PotMAX = 0.3*q
        Pot = [0.0] * int(xNum-1)
    # affichage du potentiel
    plt.rcParams.update(params)
    plt.subplot(1,2,1)
    plt.plot(xArr, numpy.array(Pot)/q,label='Potentiel', color='b')
    plt.title('Potentiel')
    plt.ylabel('Energy (eV)')
    plt.xlabel('Distance (m)')
    plt.xlim((xMIN,xMAX))

    return Pot


In [19]:
def Schroedinger_Ind(Pot,nEn,nPsi):
    """"Args:  
         - Pot: potential, 
         - nEn: niveaux d'énergie
         - nPsi: fonctions d'ondes correspondantes aux niveaux d'énergies
          Return:
            - plot the les niveaux énergétiques et les fonctions d'ondes"""
    
    # C) Calcul des fonctions d'ondes par différences finies ####################################################

    # 1. Définition des matrices A et B
    diag2 = [1.0] * int(xNum-2)
    diag3 = [1.0] * int(xNum-3)

    # Energie cinétique A
    matDiff = -2*numpy.diag(diag2,k=0) + numpy.diag(diag3,k=1) + numpy.diag(diag3,k=-1)
    # hbeV or hb ?????
    matKin = -(hb**2)/(2*m) * matDiff/(dx**2)

    # Energie potentiel B
    matPot  = numpy.diag(Pot[1:int(xNum-1)],k=0)

    # 2. Hamiltonien de l'equation de Schroedinger H = (a A + B ) with a = -(hb**2)/(2* m * dx**2) 
    H = matKin + matPot

    # 3. Résolution de l'équation aux valeurs propres H Psi = E Psi Psi[:,i]`` is the normalized eigenvector corresponding to the
    #   eigenvalue ``E[i]``.
    E, Psi = numpy.linalg.eig(H)

    # Tri en ordre croissant des valeurs propres et des vecteurs d'ondes correspondants
    E_sorted = numpy.sort(E)
    sorted_Psi = Psi[:,numpy.argsort(E)]
    # Affichage des premiers niveaux energétiques
    vectEn = numpy.arange(0,nEn,1)
    
    for ii in vectEn:
        plt.plot(xArr,[E_sorted[ii]/q]*len(xArr),label='E' + str(ii))
 
    plt.title('Niveau d\'énergie')
    plt.legend()
    plt.ylabel('Energy (eV)')
    plt.xlabel('Distance (m)')
    plt.xlim((xMIN,xMAX))


    # Fonctions d'ondes
    # ajouter Phi_0 and Phi_n = 0
    Psi_new = numpy.insert(sorted_Psi,0,0,axis=1)
    Psi_new = numpy.insert(Psi_new,xNum-1,0,axis=1)
    # norme au carré de Psi 
    PSI2 = numpy.conj(Psi_new)*Psi_new
    PSIMAX = numpy.max(PSI2)
    #affichage des fonctions d'ondes
    vectPSIn = numpy.arange(0,nPsi,1)
    plt.subplot(1,2,2)
    for ii in vectPSIn:
        plt.plot(xArr[1:None],PSI2[:,ii+1]/PSIMAX,label='E' + str(ii))

    plt.title('Fonctions d\'ondes')
    plt.legend()
    plt.ylabel('Densité de probabilité')
    plt.xlabel('Distance (m)')
    plt.xlim((xMIN,xMAX))
    plt.tight_layout()
    plt.show()

    


In [20]:
def func(potentiel,hauteur,largeur,raideur,periode,nEn,nPsi):
    """Graph with interactive widgets: computing part"""
    pot = Potentiel(idxPot=potentiel,PotMAX=hauteur,Width=largeur,K=raideur,n=periode)
    Schroedinger_Ind(pot,nEn,nPsi)

In [21]:


"""Graph with interactive widgets"""


potentiel = widgets.Select(
    options=[('1. Plat (= puits infini) -- pas de variable',1),('2. Marche (hauteur (eV)) ',2),('3. Puits fini centré sur x = 0 (hauteur (eV) et largeur (nm))',3),('4. Harmonique (raideur en N/M)',4),('5. Barrière centrée sur x = 0 (hauteur (eV) et largeur (nm))',5),('6. Sinus (periodes et hauteur (eV))',6)],
    disabled=False,
    layout=Layout(width='350px', height='150px'))

hauteur = widgets.FloatSlider(value=0.5,min=0.0001, max=2, step=0.001, continuous_update=False,orientation='vertical')
largeur = widgets.FloatSlider(value=2,min=0, max=10, step=0.1, continuous_update=False,orientation='vertical')
raideur = widgets.FloatSlider(value=0.2,min=0.01, max=1, step=0.01, continuous_update=False,orientation='vertical')
periode = widgets.IntSlider(value=4,min=1, max=10, step=1, continuous_update=False,orientation='vertical')

nEn = widgets.IntSlider(value=4,min=1, max=10, step=1, continuous_update=False,orientation='vertical')
nPsi = widgets.IntSlider(value=4,min=1, max=10, step=1, continuous_update=False,orientation='vertical')

box1 = widgets.VBox([Label('Choix du potentiel'),potentiel])
box2 = widgets.VBox([Label('Hauteur (eV)'),hauteur])
box3 = widgets.VBox([Label('Largeur (nm)'),largeur])
box4 = widgets.VBox([Label('Raideur (N/m)'),raideur])
box5 = widgets.VBox([Label('Periode (entier)'),periode])
box6 = widgets.VBox([Label('# niveaux d\'énergie'),nEn])
box7 = widgets.VBox([Label('# fonctions d\'onde'),nPsi])

ui = widgets.HBox([box1, box2, box3, box4, box5, box6, box7])

out = widgets.interactive_output(func,{'potentiel':potentiel,'hauteur':hauteur,'largeur':largeur,'raideur':raideur,'periode':periode,'nEn':nEn,'nPsi':nPsi})

display(ui,out)






Output()